1. 問題の分析

* 実行環境メモ

  * Language/Runtime: JavaScript (Node.js v22.14.0)
  * Module: CommonJS
  * 外部ライブラリ: 不使用（Node 標準も未使用）

---

### 競技プログラミング視点での分析

* 問題: `1..n` の連続した値をちょうど一度ずつ使って構成できる「構造的に異なる BST」の個数を求める。
* 制約: `1 <= n <= 19` とかなり小さい。
* 典型解: **カタラン数 (Catalan number)**。

  * `dp[n] = Σ dp[left] * dp[right]`
  * ここで `left + right = n - 1`（根を 1 つ使うため）。
* 時間計算量目標: `O(n^2)` で十分間に合う。
* ナイーブ再帰は指数時間で不必要。閉形式（組合せ）もあるが、`n=19` なら DP の方が実装も安全で読みやすい。

---

### 業務開発視点での分析

* ロジック自体はビジネスロジックとしてもシンプルで、**ボトムアップ DP** で実装するのが最も読みやすくバグも出にくい。
* 関数は純粋関数（Pure）として実装し、副作用を持たせない。
* 入力検証:

  * 型チェック: `typeof n === 'number'` かつ `Number.isFinite(n)`
  * 整数チェック: `Number.isInteger(n)`
  * 範囲チェック: `1 <= n <= 19` 以外は `RangeError`
* 例外ポリシー:

  * 型がおかしい場合 → `TypeError`
  * 範囲外の場合 → `RangeError`
* メンテナンス性:

  * 変数名は `dp`, `nodes`, `left`, `right` のように意味が分かる名前。
  * カタラン数であることをコメントに残しておくと、あとで読む人も理解しやすい。

---

### JavaScript 特有の考慮点

* **V8 最適化**

  * `Number` の単型配列を固定長で `new Array(n + 1)` 生成し、インデックスアクセスのみ実行。
  * ループは `for` を使用 (`for (let i = 0; i <= n; i++)`)。
  * `dp` に数値以外を入れないことで hidden class / element kind を安定させる。
* **GC 対策**

  * 1 回の呼び出しで生成するのは `dp` 配列 1 個だけ。
  * ループ内では新しい配列やオブジェクトを生成しない。
* **配列操作**

  * `push` すら使わず、インデックス代入のみ (`dp[i] = value`)。
  * `shift/unshift` や `sort` などコストの高い操作は不要。

---

2. アルゴリズムアプローチ比較

| アプローチ                | 時間計算量 | 空間計算量 | JS実装コスト | 可読性 | 備考                                          |
| -------------------- | ----- | ----- | ------- | --- | ------------------------------------------- |
| 方法A: ボトムアップDP（カタラン数） | O(n²) | O(n)  | 低       | 高   | 2重ループで `dp[i] = Σ dp[left] * dp[right]` を計算 |
| 方法B: 組合せの閉形式（カタラン数）  | O(n)  | O(1)  | 中       | 中   | `C_n = (1/(n+1)) * binom(2n, n)` を逐次計算      |
| 方法C: 再帰 + メモ化        | O(n²) | O(n)  | 中       | 中   | 概念的には分かりやすいがコールスタックが不要に深くなる                 |

---

3. 選択したアルゴリズムと理由

* **選択したアプローチ: 方法A – ボトムアップ DP（カタラン数）**
* **理由**

  * 制約が小さい (`n <= 19`) ため `O(n²)` で十分高速。
  * `dp[i]` の定義が明確で、誰が見ても理解しやすい。
  * 配列 1 本だけで完結し、空間効率も良い。
* **JavaScript 特有の最適化ポイント**

  * `dp` を `new Array(n + 1)` で確保し、`for` ループで添え字アクセスのみにする。
  * ホットパスに入る前に入力検証を済ませることで、ループ中に例外が投げられないようにする。
  * `Number` のみの単型配列を維持して V8 の最適化を邪魔しない。

---

4. コード実装（solution.js）

* モジュール形式: **CommonJS**
* LeetCode 想定シグネチャ: `var numTrees = function(n) { ... }`

```javascript
'use strict';

/**
 * LeetCode: Unique Binary Search Trees
 *
 * n 個の異なる値 (1..n) から作ることのできる
 * 「構造的に異なる」二分探索木 (BST) の個数を返す。
 *
 * 本質的には n 番目のカタラン数を求める問題。
 *
 * @param {number} n - ノード数 (1 <= n <= 19)
 * @returns {number} 構造的に異なる BST の個数
 *
 * @throws {TypeError} n が number でない / 有限でない / 整数でない場合
 * @throws {RangeError} n が 1〜19 の範囲外の場合
 *
 * @description
 * 時間計算量: O(n^2)
 * 空間計算量: O(n)
 */
var numTrees = function (n) {
  // ---- 入力検証（ホットパス前に早期チェック） ----
  if (typeof n !== 'number' || !Number.isFinite(n)) {
    throw new TypeError('n must be a finite number');
  }
  if (!Number.isInteger(n)) {
    throw new TypeError('n must be an integer');
  }
  if (n < 1 || n > 19) {
    throw new RangeError('n must be between 1 and 19');
  }

  // ---- DP 配列初期化 ----
  // dp[i] = i 個のノードから作れる BST の個数
  const dp = new Array(n + 1);

  // 空木 (0 個) の場合は 1 通り（「何もない」1パターン）
  dp[0] = 1;

  // 1〜n 個のノード数について順に計算
  for (let nodes = 1; nodes <= n; nodes++) {
    let count = 0;

    // 根に置くノードを 1..nodes として全探索
    // 左部分木に (root - 1) 個、右部分木に (nodes - root) 個のノード
    for (let root = 1; root <= nodes; root++) {
      const leftCount = dp[root - 1];
      const rightCount = dp[nodes - root];
      count += leftCount * rightCount;
    }

    dp[nodes] = count;
  }

  return dp[n];
};

// CommonJS 環境（ローカル検証 / CI 用）
// LeetCode 上では無視される想定
if (typeof module !== 'undefined' && module.exports) {
  module.exports = { numTrees };
}

Analyze Complexity
Runtime 0 ms
Beats 100.00%
Memory 54.74 MB
Beats 19.31%

```

---

5. 追加メモ（JS最適化チェックリスト）

* ループはすべて `for` を使用し、`forEach` / `map` などの高階関数は不使用。
* 一時オブジェクト・配列の生成は `dp` 1 本のみ。
* `dp` は数値のみを格納する単型配列で、hidden class / element kind を安定化。
* 例外はループに入る前の入力検証だけで投げる設計。
* `n <= 19` なので `Number` でオーバーフローも誤差も問題なし（最大値 `dp[19] = 1767263190`）。

結論から言うと、

* **アルゴリズム的に 0ms / O(1) 追加メモリまで既にほぼ最適**
* **LeetCode の “54MB / 19%” はほぼ環境オーバーヘッドで、コードではほとんどどうしようもない**

…なんですが、「理論上のメモリ効率」を上げる余地は少しだけあります。

---

## 1. なぜ Memory が悪く見えるのか？

* `n <= 19` なので、`dp` 配列は **長さ 20 の Number 配列** だけです。
* この配列自体はせいぜい数百バイト〜数 KB レベル。
* LeetCode の Memory 計測は、

  * JS エンジンのオーバーヘッド
  * ランタイム / テストハーネス
  * などを含んだ「プロセス単位の使用メモリ」に近く、
    **あなたのコードの数十バイト〜数 KB の差はほとんどノイズ扱い** になります。

つまり、DPで O(n) の配列を 1 個持ったからといって、メモリ使用量が 54MB → 30MB みたいに劇的に下がることはありません。

---

## 2. 「理論上」もう少し削るなら：O(1) メモリ（カタラン閉形式）

DP を使わず、**カタラン数の漸化式**でそのまま計算すれば、
「追加メモリは定数（変数数個だけ）」になります。

有名な式：

> C₀ = 1
> Cₙ = Cₙ₋₁ × 2(2n − 1) / (n + 1)

これを使うと、**配列 `dp` が不要**になります。

### 改善版（理論上 O(1) メモリ）の実装例

LeetCode 形式 + これまでのポリシー（Pure / JSDoc / 例外）を維持した形です。

```javascript
'use strict';

/**
 * LeetCode: Unique Binary Search Trees
 *
 * n 個の異なる値 (1..n) から作ることのできる
 * 「構造的に異なる」二分探索木 (BST) の個数を返す。
 *
 * 本質的には n 番目のカタラン数 C_n を求める問題。
 * ここでは C_n = C_{n-1} * 2(2n-1)/(n+1) の漸化式を利用し、
 * 追加メモリ O(1) で計算する。
 *
 * @param {number} n - ノード数 (1 <= n <= 19)
 * @returns {number} 構造的に異なる BST の個数
 *
 * @throws {TypeError} n が number でない / 有限でない / 整数でない場合
 * @throws {RangeError} n が 1〜19 の範囲外の場合
 *
 * @description
 * 時間計算量: O(n)
 * 空間計算量: O(1)
 */
var numTrees = function (n) {
  // ---- 入力検証（ホットパス前に早期チェック）----
  if (typeof n !== 'number' || !Number.isFinite(n)) {
    throw new TypeError('n must be a finite number');
  }
  if (!Number.isInteger(n)) {
    throw new TypeError('n must be an integer');
  }
  if (n < 1 || n > 19) {
    throw new RangeError('n must be between 1 and 19');
  }

  // ---- カタラン数 C_n を漸化式で計算 (C_0 = 1) ----
  // C_n = C_{n-1} * 2(2n - 1) / (n + 1)
  let c = 1; // C_0

  for (let i = 1; i <= n; i++) {
    c = (c * 2 * (2 * i - 1)) / (i + 1);
  }

  // n <= 19 の範囲では C_n は 2^53 未満 なので
  // IEEE754 の倍精度浮動小数で誤差なく表現できる。
  // 一応安全のため切り捨て/丸めしてもよい（コストはほぼ無視できる）。
  return c;
};

// CommonJS 環境（ローカル検証 / CI 用）
if (typeof module !== 'undefined' && module.exports) {
  module.exports = { numTrees };
}

Analyze Complexity
Runtime 0 ms
Beats 100.00%
Memory 53.24 MB
Beats 88.03%

```

### このアプローチのポイント

* **時間計算量**: `O(n²)` → `O(n)`（とはいえ n<=19 なので体感差はゼロ）
* **空間計算量**: `O(n)` → `O(1)`（理論上）
* JS 的にも:

  * 配列がなく、単に `Number` 変数 `c` を更新するだけ
  * hidden class/element kind の心配も不要
  * 生成オブジェクト数がさらに減る

LeetCode のメモリランキングが劇的に上がる保証はありませんが、
**アルゴリズム理論上は最もメモリ効率の良い形**になっています。

---

## 3. 実務・競技的なまとめ

* すでに Runtime 0ms (100%) なので、速度面は「これ以上やることはない」です。
* メモリは計測の特性上、「環境オーバーヘッド」が支配的で、
  コード変更で大きく上下させるのはほぼ不可能です。
* それでもこだわるなら：

  * DP 配列をやめてカタラン数の漸化式で O(1) メモリにする（上の実装）。
  * LeetCode 用に限れば、入力バリデーションを削ると（ほんの少しだけ）クリーンなコード＆わずかなメモリ削減になるかもしれません。

なので、

* **アルゴリズム・実装品質としてはすでに十分トップレベル**
* 趣味・学習として「理論上も最適」を目指すなら、上のカタラン漸化式版に差し替え

くらいの認識でOKです。